In [40]:
import warnings
warnings.filterwarnings(action='ignore')
import re
import pymongo
import pandas as pd
from konlpy.tag import Okt
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import datasets, layers, models, optimizers

In [2]:
conn = pymongo.MongoClient()
conn

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [3]:
chatbot = conn.chatbot
chatbot

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'chatbot')

In [4]:
QnA = chatbot.QnA
QnA

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'chatbot'), 'QnA')

In [5]:
QnA_list = pd.DataFrame()
docs = QnA.find({})
for doc in docs:
    df = pd.DataFrame(docs, columns={'SPEAKER', 'SENTENCE', 'MAIN'})
    QnA_list = QnA_list.append(df)
QnA_list

,MAIN,SENTENCE,SPEAKER
0,배송 문의,"네, 가능합니다.",점원
1,배송 문의,배송은 몇일 더 걸리나요?,고객
2,배송 문의,3일 정도 더 소요 예상됩니다.,점원
3,배송 문의,택배로도 가능한가요?,고객
4,배송 문의,네 고객님 주소알려주시면 택배로도 가능합니다,점원
...,...,...,...
203793,배송 문의,주문해주신 상품금일배송되셨습니다 배송 조회 가능하세요.,점원
203794,배송 문의,"2018.08.17배송 도착이여서 2018.08.14 구매했는데요, 아직 상품 준비...",고객
203795,배송 문의,14일 오전 9시 이후 결제건은 16일 오전 9시부터 주문 확인 시작되시고요.,점원
203796,배송 문의,지난달 23일에 주문했습니다. 이번 주에는 받을 수 있는 건가요?,고객


In [6]:
QnA_list_c = QnA_list[QnA_list['SPEAKER'] == '고객']
QnA_list_c

,MAIN,SENTENCE,SPEAKER
1,배송 문의,배송은 몇일 더 걸리나요?,고객
3,배송 문의,택배로도 가능한가요?,고객
5,배송 문의,네 그럼 택배로 부탁드려요,고객
8,배송 문의,택배는 얼마나 걸리죠?,고객
10,배송 문의,그럼 택배로 보내주세요,고객
...,...,...,...
203788,배송 문의,혹시 이번 주에는 받을 수 있을까요?,고객
203790,배송 문의,배송문의 발송 예정일 5/1로 안내받았는데 발송되나요?,고객
203792,배송 문의,오늘 출고 가능한가요?,고객
203794,배송 문의,"2018.08.17배송 도착이여서 2018.08.14 구매했는데요, 아직 상품 준비...",고객


In [7]:
# 배송 문의 값 제거
idx = QnA_list_c[QnA_list_c['MAIN']=='배송 문의'].index
QnA_list_c.drop(idx, inplace=True)
QnA_list_c['MAIN'].value_counts()

의류 문의    49153
상품 추천    11359
구매 목록     3456
Name: MAIN, dtype: int64

In [8]:
QnA_list_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63968 entries, 16 to 128493
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   MAIN      63968 non-null  object
 1   SENTENCE  63968 non-null  object
 2   SPEAKER   63968 non-null  object
dtypes: object(3)
memory usage: 2.0+ MB


## 토큰화(불용어 제거)

In [9]:
#한글과 공백을 제외하고 모두 제거
QnA_list_c['SENTENCE'] = QnA_list_c['SENTENCE'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ가-힣]","")
QnA_list_c.tail()

,MAIN,SENTENCE,SPEAKER
128485,구매 목록,가 방색 상 변경이 가능할까요,고객
128487,구매 목록,번 셔츠 사이즈 주문했으니 사이즈 잘 확인하고 보내주세요 그리고 혹시 셔츠가 슬림...,고객
128489,구매 목록,일자로 주문했는데 스키니로 주문 변경하고 싶은데요,고객
128491,구매 목록,면 사이즈가 큰데 로 가능할까요,고객
128493,구매 목록,선택에는 없는데 분홍과 파랑 족씩 구매하고 싶은데 가능한가요,고객


In [10]:
#전처리 후 테스트용 샘플의 개수
print(len(QnA_list_c['SENTENCE']))

63968


In [11]:
# stopwords 명시
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords = sum(stopwords, [])

In [12]:
okt = Okt()

In [13]:
#불용어제거
data1 = []
for sentence in tqdm(QnA_list_c['SENTENCE']):
    tokenized_sentence = okt.morphs(sentence, stem=True)#토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence
                                  if not word in stopwords]#불용어제거
    data1.append(stopwords_removed_sentence)

100%|███████████████████████████████████████████████████████████████████████████| 63968/63968 [01:08<00:00, 932.51it/s]


In [14]:
QnA_token = data1

# QnA_token을 fit 시켜보자

In [15]:
tokenizer = Tokenizer(4729)
tokenizer.fit_on_texts(QnA_token)

In [16]:
print(tokenizer.word_index)
print(tokenizer.word_counts.items())

{'하다': 1, '사이즈': 2, '요': 3, '는': 4, '되다': 5, '은': 6, '거': 7, '인가요': 8, '어떻다': 9, '번': 10, '입다': 11, '맞다': 12, '도': 13, '제품': 14, '없다': 15, '수': 16, '건가': 17, '바지': 18, '게': 19, '나오다': 20, '안': 21, '주문': 22, '상품': 23, '가요': 24, '추천': 25, '보다': 26, '좋다': 27, '인데': 28, '예요': 29, '뭐': 30, '허리': 31, '그렇다': 32, '색상': 33, '죠': 34, '한': 35, '해주다': 36, '면': 37, '옷': 38, '중': 39, '자다': 40, '아니다': 41, '크다': 42, '정도': 43, '줄다': 44, '에요': 45, '구매': 46, '만': 47, '인': 48, '많이': 49, '색': 50, '괜찮다': 51, '더': 52, '않다': 53, '랑': 54, '키': 55, '하나요': 56, '이다': 57, '가능하다': 58, '네네': 59, '아': 60, '작다': 61, '되어다': 62, '걸': 63, '가슴': 64, '원피스': 65, '입': 66, '둘레': 67, '브라': 68, '가방': 69, '알다': 70, '용': 71, '남자': 72, '할인': 73, '이에요': 74, '다': 75, '색깔': 76, '블랙': 77, '길이': 78, '기모': 79, '돼다': 80, '디자인': 81, '보여주다': 82, '즈': 83, '이건': 84, '싶다': 85, '선택': 86, '이랑': 87, '너무': 88, '평소': 89, '모델': 90, '제일': 91, '데': 92, '세': 93, '들다': 94, '어울리다': 95, '사다': 96, '건': 97, '오다': 98, '이즈': 99, '차이': 100, '뭔가': 101, '하고': 1

## text_sequence(글자 길이 맞추기)

In [17]:
total_cnt = len(tokenizer.word_index)
total_cnt

10557

In [18]:
tokenizer.fit_on_texts(data1)
data1 = tokenizer.texts_to_sequences(data1)

In [19]:
data1[0], data1[10], data1[100]

([216, 2185, 171, 13, 27, 32, 97, 15], [120, 161, 337, 9, 19, 40, 394], [9, 7])

In [20]:
print('질문 평균 길이: ', sum(map(len,data1))/len(data1))

질문 평균 길이:  5.9613869434717355


In [21]:
data1[:10]

[[216, 2185, 171, 13, 27, 32, 97, 15],
 [],
 [377, 6, 50, 87, 40, 95],
 [65, 4, 9, 19, 258],
 [1091, 264, 11, 21, 30, 11, 27],
 [116, 1000, 696, 19, 95, 3],
 [13, 50, 91, 27, 7],
 [57, 38],
 [7, 54, 268, 103, 6, 30],
 []]

# padding

In [22]:
#전체 데이터 길이는 7로 맞춘다.
max_len = 10
data1_one_hot = pad_sequences(data1, maxlen = max_len, padding='post')
data1_one_hot[:5]

array([[ 216, 2185,  171,   13,   27,   32,   97,   15,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 377,    6,   50,   87,   40,   95,    0,    0,    0,    0],
       [  65,    4,    9,   19,  258,    0,    0,    0,    0,    0],
       [1091,  264,   11,   21,   30,   11,   27,    0,    0,    0]])

# train, test data 분류

## 라벨 인코딩

In [23]:
data_y = QnA_list_c['MAIN']
data_y

16        상품 추천
18        상품 추천
19        상품 추천
21        상품 추천
23        상품 추천
          ...  
128485    구매 목록
128487    구매 목록
128489    구매 목록
128491    구매 목록
128493    구매 목록
Name: MAIN, Length: 63968, dtype: object

In [24]:
# 라벨 인코딩(카테고리형 문자를 숫자로 자동 변환해주는 역할)
y = np.array(data_y)
encoder = LabelEncoder()
label = encoder.fit_transform(data_y)
label[:5]

array([1, 1, 1, 1, 1])

In [25]:
encoder.classes_

array(['구매 목록', '상품 추천', '의류 문의'], dtype=object)

In [26]:
label.astype('float')

array([1., 1., 1., ..., 0., 0., 0.])

In [27]:
x = data1_one_hot
y_label = label
X_train, X_test, y_train, y_test = train_test_split(x,
                                                    y_label,
                                                    test_size= 0.2,
                                                    random_state=55
                                                   )
print(X_train.shape, X_test.shape)

(51174, 10) (12794, 10)


# 단어 집합의 크기 구하기
## 집합의 크기로 data를 fit시키기

In [28]:
threshold = 5
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq= total_freq + value
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value
vocab_size= total_cnt-rare_cnt + 1
print("단어의 집합 크기:", vocab_size)

단어의 집합 크기: 4729


# LSTM으로 분류 시작

In [29]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h6', monitor='val_acc', mode='max', verbose=1,
                    save_best_only=True)

In [30]:
model = Sequential()
model.add(Embedding(vocab_size, 16 ,input_length=10))
model.add(LSTM(8, dropout=0.3))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 16)            75664     
                                                                 
 lstm (LSTM)                 (None, 8)                 800       
                                                                 
 dense (Dense)               (None, 3)                 27        
                                                                 
Total params: 76,491
Trainable params: 76,491
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
             metrics=['acc'])

In [32]:
X_train[0]

array([677, 337,  47,  35, 186,  25,  13,  36,  16,   0])

In [33]:
history = model.fit(X_train,
                    y_train,
                    epochs=50,
                    callbacks=[es, mc],
                    batch_size= 80,
                    validation_split=0.2
                    )

Epoch 1/50
510/512 [============================>.] - ETA: 0s - loss: 0.5772 - acc: 0.8000
Epoch 00001: val_acc improved from -inf to 0.83361, saving model to best_model.h6


INFO:tensorflow:Assets written to: best_model.h6\assets


INFO:tensorflow:Assets written to: best_model.h6\assets


512/512 [==============================] - 19s 33ms/step - loss: 0.5767 - acc: 0.8000 - val_loss: 0.4167 - val_acc: 0.8336
Epoch 2/50
502/512 [============================>.] - ETA: 0s - loss: 0.3837 - acc: 0.8545
Epoch 00002: val_acc improved from 0.83361 to 0.86273, saving model to best_model.h6


INFO:tensorflow:Assets written to: best_model.h6\assets


...

INFO:tensorflow:Assets written to: best_model.h6\assets


512/512 [==============================] - 15s 29ms/step - loss: 0.3236 - acc: 0.8735 - val_loss: 0.3474 - val_acc: 0.8634
Epoch 5/50
507/512 [============================>.] - ETA: 0s - loss: 0.3094 - acc: 0.8775
Epoch 00005: val_acc did not improve from 0.86341
512/512 [==============================] - 3s 5ms/step - loss: 0.3094 - acc: 0.8773 - val_loss: 0.3477 - val_acc: 0.8612
Epoch 6/50
510/512 [============================>.] - ETA: 0s - loss: 0.3025 - acc: 0.8798
Epoch 00006: val_acc improved from 0.86341 to 0.86546, saving model to best_model.h6


INFO:tensorflow:Assets written to: best_model.h6\assets


INFO:tensorflow:Assets written to: best_model.h6\assets


512/512 [==============================] - 13s 25ms/step - loss: 0.3024 - acc: 0.8798 - val_loss: 0.3496 - val_acc: 0.8655
Epoch 7/50
510/512 [============================>.] - ETA: 0s - loss: 0.2940 - acc: 0.8794
Epoch 00007: val_acc did not improve from 0.86546
512/512 [==============================] - 3s 5ms/step - loss: 0.2940 - acc: 0.8794 - val_loss: 0.3495 - val_acc: 0.8627
Epoch 8/50
512/512 [==============================] - ETA: 0s - loss: 0.2900 - acc: 0.8816
Epoch 00008: val_acc did not improve from 0.86546
512/512 [==============================] - 3s 5ms/step - loss: 0.2900 - acc: 0.8816 - val_loss: 0.3483 - val_acc: 0.8637
Epoch 9/50
509/512 [============================>.] - ETA: 0s - loss: 0.2837 - acc: 0.8841
Epoch 00009: val_acc did not improve from 0.86546
512/512 [==============================] - 3s 5ms/step - loss: 0.2836 - acc: 0.8842 - val_loss: 0.3495 - val_acc: 0.8626
Epoch 00009: early stopping


In [34]:
X_train.shape, y_train.shape

((51174, 10), (51174,))

In [35]:
model2=models.load_model('./best_model.h6/')
model2

In [36]:
model2.evaluate(X_test, y_test)[1]

400/400 [==============================] - 1s 1ms/step - loss: 0.3561 - acc: 0.8641


0.8640769124031067

In [51]:
def category_predict(new_sentence):
    new_sentence = re.sub(r"[^ㄱ-ㅎ ㅏ-ㅣ가-힣]","", new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True)#토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords]#불용어제거
    encoded = tokenizer.texts_to_sequences([new_sentence])#정수인코딩
    pad_new = pad_sequences(encoded, maxlen=max_len)#패딩
    score = model2.predict(pad_new)#예측
    #상품추천:1
    #구매목록:0
    #의류문의:2
    out_x = np.where(score<0.5, "구매목록 내용",(1.5 < score,"상품추천이군요~","의류문의군요~"))
    out_category = np.argmax(out_x)
    if out_category == 0:
        out_category = "구매 목록"
    elif out_category == 1:
        out_category = "상품 추천"
    elif out_category == 2:
        out_category = "의류 문의"
    else:
        pass

    print(out_category + "애 대해 질문하셨군요!")
#     if(1.5 < score):
#         print("상품추천에 대한 내용")
#     elif(score < 0.5):
#         print("구매목록에 대한 내용")
#     else:
#         print("의류문의군요~")

In [52]:
category_predict("블라우스 추천해줘!")

상품 추천애 대해 질문하셨군요!


In [53]:
category_predict("이 옷이 뭔지 모르겠어.. 알려주지 않을래?")

의류 문의애 대해 질문하셨군요!


In [54]:
category_predict("내가 구매한 구매목록 보여줘")

구매 목록애 대해 질문하셨군요!
